# Packages

In [76]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ambrosia.designer import Designer

# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {day} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    for date in daterange:
        print(f"Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')


# First iteration. Test launch in small cities
-------

### Avatars & Nicknames

#### Nicnames

In [ ]:
df_nickname = read_bq("""
WITH cte AS (SELECT *
             FROM (SELECT user_id,
                          username,
                          COUNT(*) OVER (PARTITION BY user_id) cnt
                   FROM (SELECT id AS         user_id,
                                username,
                                MIN(modified) modified
                         FROM dwh-storage-327422.personal_data.dir_user_hst
                         WHERE start_dt_part > '2022-08-01'
                           AND date(modified) >= date_add(current_date(), interval - 30 day)
                           AND mode = 'client'
                         GROUP BY 1, 2) _) __
            -- WHERE cnt > 1
             ORDER BY 1)
SELECT *
FROM cte t1
         JOIN (SELECT id,
                      city_id
               FROM indriver-e6e40.ods_monolith.tbl_user
               WHERE mode = 'client') t2
              ON t1.user_id = t2.id
                  AND city_id IN (4228, 4261, 4269, 4517, 833)
""")

df_nickname.head()

In [3]:
df_nickname_agg = df_nickname.groupby(['user_id', 'city_id'], as_index=False)['cnt'].agg('max')

def cities(row):
    if row == 833:
         return 'Durban'

    elif row == 4517:
        return 'Natal'

    elif row == 4269:
        return 'Puerto Montt'

    elif row == 4261:
        return 'Concepcion'

    elif row == 4228:
        return 'Tijuana'

df_nickname_agg['cities'] = df_nickname_agg['city_id'].apply(cities)



In [ ]:
percentile = 95

for city in df_nickname_agg['cities'].unique():

    lower, upper = st.t.interval(confidence=0.95, df=len(df_nickname_agg.query(f"cities == '{city}'")['cnt'])-1, 
              loc=np.percentile(df_nickname_agg.query(f"cities == '{city}'")['cnt'], percentile), 
              scale=st.sem(df_nickname_agg.query(f"cities == '{city}'")['cnt']))

    result = int(np.percentile(df_nickname_agg.query(f"cities == '{city}'")['cnt'], percentile))

    fig = px.histogram(df_nickname_agg.query(f"cities == '{city}'")['cnt']
                , x="cnt"
                , title=f"Number of users with nicknames in {city} / {lower:.2f} and {upper:.2f}"
                , nbins=60
                , text_auto=True               )
    fig.update_traces(textposition='inside')

    fig.update_layout(
        xaxis_title="Number of completed orders"
        , yaxis_title="Number of records"
    )

    fig.add_vline(x=result, line_width=2, line_dash="dash", line_color="black", annotation_text=f' {percentile}% of users with following number of rides {result} ')

    fig.update_xaxes(dtick=1)


    fig.show()

#### Avatars

In [5]:
df_avatars = read_bq("""
WITH cte AS (SELECT id,
                    country_id,
                    city_id,
                    created,
                    modified,
                    mode,
                    type,
                    avatarmedium,
                    gender,
                    username,
             FROM dwh-storage-327422.personal_data.dir_user_hst
             WHERE start_dt_part >= '2024-08-01'
               AND created >= '2024-08-01'
--                AND id = 234811634
               AND mode = 'client')
SELECT id, avatarmedium, MIN(modified)
FROM cte
WHERE date(modified) >= date_add(current_date(), interval - 30 day)
--   AND id = 234811634
GROUP BY 1, 2
-- QUALIFY ROW_NUMBER() OVER (PARTITION BY id) > 2
ORDER BY id
""")

df_avatars.head()

df_avatars_agg = df_avatars.groupby('id', as_index=False)['avatarmedium'].agg('count')

In [ ]:
percentile = 90

lower, upper = st.t.interval(confidence=0.95, df=len(df_avatars_agg['avatarmedium'])-1, 
        loc=np.percentile(df_avatars_agg['avatarmedium'], percentile), 
        scale=st.sem(df_avatars_agg['avatarmedium']))

result = int(np.percentile(df_avatars_agg['avatarmedium'], percentile))

fig = px.histogram(df_avatars_agg['avatarmedium']
        , x="avatarmedium"
        , title=f"Number of users with features number of rides / {lower:.2f} and {upper:.2f}"
        , nbins=100
        , text_auto=True               )
fig.update_traces(textposition='inside')

fig.update_layout(
xaxis_title="Number of completed orders"
, yaxis_title="Number of records"
)

fig.add_vline(x=result, line_width=2, line_dash="dash", line_color="black", annotation_text=f' {percentile}% of users with following number of rides {result} ')

fig.update_xaxes(dtick=1)


fig.show()

In [ ]:
df_avatar_2 = read_bq("""
WITH cte AS (SELECT id,
                    country_id,
                    city_id,
                    created,
                    modified,
                    mode,
                    type,
                    avatarmedium,
                    gender,
                    username,
             FROM dwh-storage-327422.personal_data.dir_user_hst
             WHERE start_dt_part >= '2024-07-01'
               AND created >= '2024-07-01'
               AND mode = 'client'),
     cities AS (SELECT t1.id,
                       CAST(t1.created AS date)      AS reg_date,
                       DATE_TRUNC(t1.created, MONTH) AS month,
                       t1.mode,
                       t1.country_id,
                       t2.country_name,
                       t2.macroregion_name,
                       t1.city_id,
                       t2.city_name,
                       CASE
                           WHEN osversion LIKE '%android%' THEN 'android'
                           WHEN osversion LIKE '%ios%' THEN 'ios'
                           ELSE
                               NULL
                           END
                                                        os
                FROM indriver-e6e40.ods_monolith.tbl_user t1
                         JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                              ON t1.city_id = t2.city_id
                WHERE 1 = 1
                  AND CAST(t1.created AS DATE) >= '2024-07-01'
                  AND t1.mode = 'client'
                QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.id ORDER BY t1.created DESC) = 1)
SELECT t1.id,
       t2.country_name,
       avatarmedium,
       IF(avatarmedium IS NULL, 'no_photo', avatarmedium) AS photo,
       MIN(modified)
FROM cte t1
         JOIN cities t2
              ON t1.id = t2.id
WHERE date(modified) >= date_add(current_date(), interval - 30 day)
AND t2.country_id IN (25, 12, 11, 10)
GROUP BY 1, 2, 3, 4
ORDER BY id
""")

df_avatar_2.head()

df_avatar_2_agg = df_avatar_2.groupby(['id', 'country_name'], as_index=False)['avatarmedium'].agg('count')

df_avatar_2_agg.pivot_table(values='id', index='country_name',
                       columns='avatarmedium', aggfunc="count")
brazil = [1157025,381634,15386,1652,328]
chile = [115952,178577,6989,732,121]
mexico = [1164598,306897,15164,1848,429]
south_africe = [309785,151282,5291,471,87]
print(381634/np.sum(brazil)*100)
print(6989/np.sum(chile)*100)
print(306897/np.sum(mexico)*100)
print(151282/np.sum(south_africe)*100)

percentile = 90

cities = ['Chile', 'Mexico', 'Brazil', 'South Africa']

df_avatar_2_agg = df_avatar_2.groupby(['id', 'country_name'], as_index=False)['avatarmedium'].agg('count')

for city in cities:

        lower, upper = st.t.interval(confidence=0.95, df=len(df_avatar_2_agg.query(f"country_name == '{city}'")['avatarmedium'])-1, 
                loc=np.percentile(df_avatar_2_agg.query(f"country_name == '{city}'")['avatarmedium'], percentile), 
                scale=st.sem(df_avatar_2_agg.query(f"country_name == '{city}'")['avatarmedium']))

        result = int(np.percentile(df_avatar_2_agg.query(f"country_name == '{city}'")['avatarmedium'], percentile))

        fig = px.histogram(df_avatar_2_agg.query(f"country_name == '{city}'")['avatarmedium']
                , x="avatarmedium"
                , title=f"In {city} / {lower:.2f} and {upper:.2f}"
                , nbins=50
                , text_auto=True               )


        fig.update_traces(marker_color='blue', marker_line_color='black',
                        marker_line_width=1.5, opacity=0.5, textposition='inside')


        fig.update_layout(
        xaxis_title="Number of completed orders"
        , yaxis_title="Number of records"
        )

        fig.add_vline(x=result, line_width=2, line_dash="dash", line_color="black", annotation_text=f' {percentile}% of users with following number of rides {result} ')

        # fig.update_xaxes(dtick=1)


        fig.show()

### Completed orders

In [ ]:
### First iteration
df_completed_orders = read_bq("""
WITH cte AS (SELECT user_type,
                    user_id,
                    t1.country_id,
                    country_name,
                    t1.city_id,
                    city_name,
                    metric_date,
                    rides_count,
                    IF(rides_count > 0, 1, 0)                                                                             AS has_ride,
                    LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date)                                    AS next_ride_date,
                    SUM(rides_count)
                        OVER (PARTITION BY user_id ORDER BY metric_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum
             FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                      JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                           ON t1.city_id = t2.city_id
             WHERE metric_date >= '2024-01-01'
               AND user_type = 'pass'
               AND t1.city_id IN (4228, 4261, 4269, 4517, 833))
SELECT user_id, city_id, city_name, country_name, SUM(rides_count) AS completed_orders
FROM cte
GROUP BY 1, 2, 3, 4
""")

df_completed_orders.head()

percentile = 50

for city in df_completed_orders['city_name'].unique():

    lower, upper = st.t.interval(confidence=0.95, df=len(df_completed_orders.query(f"city_name == '{city}'")['completed_orders'])-1, 
              loc=np.percentile(df_completed_orders.query(f"city_name == '{city}'")['completed_orders'], percentile), 
              scale=st.sem(df_completed_orders.query(f"city_name == '{city}'")['completed_orders']))

    result = int(np.percentile(df_completed_orders.query(f"city_name == '{city}'")['completed_orders'], percentile))

    fig = px.histogram(df_completed_orders.query(f"city_name == '{city}'")['completed_orders']
                , x="completed_orders"
                , title=f"Number of users with features number of rides in {city} / {lower:.2f} and {upper:.2f}"
                , nbins=500                )
    fig.update_traces(textposition='inside')

    fig.update_layout(
        xaxis_title="Number of completed orders"
        , yaxis_title="Number of records"
    )

    fig.add_vline(x=result, line_width=2, line_dash="dash", line_color="black", annotation_text=f' {percentile}% of users with following number of rides {result} ')

    fig.update_xaxes(dtick=10, 
                     tickformat="%b %Y")


    fig.show()
df_completed_ord_inc = read_bq("""
WITH cte AS (SELECT user_type,
                    user_id,
                    t1.country_id,
                    country_name,
                    t1.city_id,
                    city_name,
                    metric_date,
                    rides_count,
                    IF(rides_count > 0, 1, 0)                                                                             AS has_ride,
                    LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date)                                    AS next_ride_date,
                    SUM(rides_count)
                        OVER (PARTITION BY user_id ORDER BY metric_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum
             FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                      JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                           ON t1.city_id = t2.city_id
             WHERE metric_date >= '2024-01-01'
               AND user_type = 'pass'
               AND t1.country_id IN (25, 12, 11, 10))
SELECT user_id,
       metric_date,
       t1.city_id,
       t1.city_name,
       t1.country_name,
       cum_sum,
       t2.pass_id,
       t2.incident_date
FROM cte t1
         LEFT JOIN (SELECT *
                    FROM indriver-bi.safety.vw_safety_incidents_detail
                    WHERE aggressor = 'Passenger'
                      AND incident_date >= '2024-01-01'
                      AND information_status = 'Confirmed') t2
                   ON t1.user_id = t2.pass_id
                       AND t2.incident_date = metric_date
WHERE t2.pass_id IS NOT NULL
--   AND t1.user_id = 207288762
""")

df_completed_ord_inc.head()
df_completed_ord_inc.query('cum_sum < 13').pivot_table(values='pass_id', index='country_name',
                       columns='cum_sum', aggfunc="count")
brazil = [2814,	1169,673,501,381,268,256,231,188,180,138,120,156]
chile = [347,194,104,87,64,	55,	37,	38,	29,	38,	27,	28,	29]
mexico = [612,462,252,187,147,122,74,91,56,	44,	63,	43,	43]
south_africa = [552,327,145,104,62,	57,	38,	31,	28,	25,	15,	13,	12]


print('Brazil - ', (2814+1169)/np.sum(brazil)*100)
print('Chile - ', (347+194)/np.sum(chile)*100)
print('Mexico - ', (612+462)/np.sum(mexico)*100)
print('South Africa - ', (552+327)/np.sum(south_africa)*100)
percentile = 50

for city in df_completed_ord_inc['country_name'].unique():

    lower, upper = st.t.interval(confidence=0.95, df=len(df_completed_ord_inc.query(f"country_name == '{city}'")['cum_sum'])-1, 
              loc=np.percentile(df_completed_ord_inc.query(f"country_name == '{city}'")['cum_sum'], percentile), 
              scale=st.sem(df_completed_ord_inc.query(f"country_name == '{city}'")['cum_sum']))

    result = int(np.percentile(df_completed_ord_inc.query(f"country_name == '{city}'")['cum_sum'], percentile))

    fig = px.histogram(df_completed_ord_inc.query(f"country_name == '{city}'")['cum_sum']
                , x="cum_sum"
                , title=f"Number of users with features number of rides in {city} / {lower:.2f} and {upper:.2f}"
                , nbins=500                )
    fig.update_traces(textposition='inside')

    fig.update_layout(
        xaxis_title="Number of completed orders"
        , yaxis_title="Number of records"
    )

    fig.add_vline(x=result, line_width=2, line_dash="dash", line_color="black", annotation_text=f' {percentile}% of users with following number of rides {result} ')

    fig.update_xaxes(dtick=10, 
                     tickformat="%b %Y")


    fig.show()

### Negative reviews

In [ ]:
df_avg_rating_1 = read_bq("""
WITH cte AS (SELECT t1.target_id,
                    t1.rating,
                    t1.created_date_part,
                    t1.visibility,
                    t1.country_id,
                    t1.city_id,
                    t2.country_name
             FROM indriver-e6e40.ods_review.tbl_review t1
                      JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                           ON t1.city_id = t2.city_id
             WHERE visibility = 'passenger'
               AND t1.country_id IN (25, 12, 11, 10)
               AND created_date_part >= DATE_ADD(CURRENT_DATE()
                 , INTERVAL -30 DAY)
             QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.target_id ORDER BY t1.created_at DESC) <= 3),
     rating AS
         (SELECT target_id,
                 visibility,
                 created_date_part,
                 country_id,
                 country_name,
                 ARRAY_TO_STRING(ARRAY_AGG(CAST(rating AS STRING)), ', ') AS ratings
          FROM cte
          GROUP BY 1, 2, 3, 4, 5)
SELECT *,
       CASE
           WHEN (ratings LIKE "%1%2%" OR ratings LIKE "%2%1%" OR ratings LIKE "%1%" OR ratings LIKE "%2%")
               THEN 'negative_reviews'
           ELSE 'the_rest'
           END segments
FROM rating
""")

df_avg_rating_1.head()
df_avg_rating_agg = df_avg_rating_1.groupby(['segments', 'country_name'], as_index=False)['target_id'].agg('count')

df_avg_rating_agg_country = df_avg_rating_1.groupby(['country_name'], as_index=False)['target_id'].agg('count')

df_avg_rating_agg = df_avg_rating_agg.merge(df_avg_rating_agg_country, left_on='country_name', right_on='country_name')

df_avg_rating_agg['share'] = df_avg_rating_agg['target_id_x'] / df_avg_rating_agg['target_id_y'] * 100

df_avg_rating_agg
fig = px.bar(df_avg_rating_agg, x="country_name", y="share", 
             text_auto=True,
             color='segments',
             title=''
             )


fig.update_traces(textposition='inside', textfont_size=12)
fig.update_layout(barmode='group')


fig.show()

### Days since last ride

In [ ]:
df_days = read_bq("""
SELECT user_type,
       user_id,
       t1.country_id,
       country_name,
       t1.city_id,
       city_name,
       metric_date,
       IF(rides_count > 0, 1, 0)                                              AS has_ride,
       IF(LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date) IS NULL, CURRENT_DATE(),
          LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date)) AS next,
       DATE_DIFF(IF(LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date) IS NULL,
                    CURRENT_DATE(),
                    LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date)), metric_date,
                 DAY)                                                         AS date_diff_next,
       LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date)     AS next_ride_date,
       DATE_DIFF(LEAD(metric_date) OVER (PARTITION BY user_id ORDER BY metric_date), metric_date,
                 DAY)                                                         AS date_diff
FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
         JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
              ON t1.city_id = t2.city_id
WHERE metric_date >= '2024-01-01'
  AND IF(rides_count > 0, 1, 0) = 1
  AND user_type = 'pass'
  AND t1.city_id IN (4228, 4261, 4269, 4517, 833, 5568, 5575)
""")

df_days.head()

df_days.groupby('city_name', as_index=False)['date_diff_next'].quantile(0.9)

# Each records by users with day difference

In [ ]:
percentile = 90

for city in df_days['city_name'].unique():

    lower, upper = st.t.interval(confidence=0.95, df=len(df_days.query(f"city_name == '{city}'")['date_diff_next'])-1, 
              loc=np.percentile(df_days.query(f"city_name == '{city}'")['date_diff_next'], percentile), 
              scale=st.sem(df_days.query(f"city_name == '{city}'")['date_diff_next']))

    result = int(np.percentile(df_days.query(f"city_name == '{city}'")['date_diff_next'], percentile))

    fig = px.histogram(df_days.query(f"city_name == '{city}'")['date_diff_next']
                , x="date_diff_next"
                , title=f"Number of rides with day difference in {city} / {lower:.2f} and {upper:.2f}"
                , nbins=500                )
    fig.update_traces(textposition='inside')

    fig.update_layout(
        xaxis_title="Days since last ride"
        , yaxis_title="Number of records"
    )

    fig.add_vline(x=result, line_width=2, line_dash="dash", line_color="black", annotation_text=f'{percentile}% of rides are placed on the left side with {result} day difference')

    fig.update_xaxes(dtick=10, 
                     tickformat="%b %Y")


    fig.show()

In [ ]:


percentile = 90

for city in df_days['city_name'].unique():

    result = int(np.percentile(df_days.query(f"city_name == '{city}'")['date_diff_next'], percentile))

    plt.rcParams["figure.figsize"] = (17,7.5)
    plt.xticks(np.arange(min(df_days.query(f"city_name == '{city}'")['date_diff_next']), max(df_days['date_diff_next'])+1, 2.0))

    one,two,three = plt.hist(df_days.query(f"city_name == '{city}'")['date_diff_next'],
            bins=210,
            color = "lightblue",
            ec="#ff704d")

    plt.axvline(int(np.percentile(df_days.query(f"city_name == '{city}'")['date_diff_next'], percentile)), color='black', linestyle='dashed', linewidth=1.2)

    min_ylim, max_ylim = plt.ylim()
    plt.text(int(np.percentile(df_days.query(f"city_name == '{city}'")['date_diff_next'], percentile))*1.1, max_ylim*0.9, f"{percentile}% of rides are placed on the left side with {result} day difference")


    for i in range(int(np.percentile(df_days.query(f"city_name == '{city}'")['date_diff_next'], percentile))):
        three[i].set_fc('#6666ff')

    plt.xlabel('Average days between rides') 
    plt.ylabel('Number of rides') 
    plt.title(f'Number of rides with day difference in {city}') 

    plt.bar_label(three, fontsize=8, rotation='vertical', padding=+7,
                color='blue', 
                fontweight='bold'
                )
    plt.margins(x=0.01, y=0.1)

    plt.show()

# Days since last ride 


In [44]:
df_dslr_incidents = read_bq("""
WITH incidents AS (SELECT redmine_id,
                          incident_date,
                          vertical,
                          incident_type,
                          category,
                          city_id,
                          city_name,
                          pass_id
                   FROM indriver-bi.safety.vw_safety_incidents_detail
                   WHERE aggressor = 'Passenger'
                     AND incident_date >= '2024-06-01'
                     AND information_status = 'Confirmed'),
     rides AS (SELECT DISTINCT user_type,
                               user_id,
                               t1.country_id,
                               t2.country_name,
                               t1.city_id,
                               t2.city_name,
                               metric_date_utc,
                               orders_count,
                               rides_count,
                               IF(rides_count > 0, 1, 0)                                                 AS has_ride,
                               LAG(metric_date_utc) OVER (PARTITION BY user_id ORDER BY metric_date_utc) AS lag_date,
                               DATE_DIFF(metric_date_utc,
                                         LAG(metric_date_utc) OVER (PARTITION BY user_id ORDER BY metric_date_utc),
                                         DAY)                                                            AS date_diff
               FROM (SELECT user_type,
                            user_id,
                            country_id,
                            city_id,
                            metric_date_utc,
                            SUM(orders_count) AS orders_count,
                            SUM(rides_count)  AS rides_count
                     FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
                     GROUP BY 1, 2, 3, 4, 5) t1
                        JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                             ON t1.city_id = t2.city_id
               WHERE metric_date_utc >= '2024-06-01'
                 AND IF(rides_count > 0, 1, 0) = 1
                 AND user_type = 'pass'
                 AND t1.country_id IN (11, 25, 12, 24, 22)),
     total_rides AS (SELECT country_name, user_id, ROUND(AVG(date_diff), 0) AS avg_date_diff
                     FROM rides
                     GROUP BY 1, 2)
SELECT t1.*,
       SUM(incidents) OVER (PARTITION BY t1.country_name)                                                          AS total_incidents,
       SUM(incidents)
           OVER (PARTITION BY t1.country_name ORDER BY t1.metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum_incidents,
       t2.total_users,
       SUM(total_users) OVER (PARTITION BY t1.country_name)                                                        AS total_users_by_country,
       SUM(total_users)
           OVER (PARTITION BY t1.country_name ORDER BY t1.metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum_users
FROM (SELECT country_name, metric, COUNT(redmine_id) AS incidents,
      FROM (SELECT t2.country_id,
                   t2.country_name,
                   DATE_DIFF(incident_date, metric_date_utc, DAY) AS metric,
                   redmine_id
            FROM incidents t1
                     JOIN rides t2
                          ON t1.pass_id = t2.user_id
                              AND t2.metric_date_utc < t1.incident_date
            QUALIFY ROW_NUMBER() OVER (PARTITION BY redmine_id ORDER BY metric_date_utc DESC) = 1)
      GROUP BY 1, 2) t1
         JOIN (SELECT country_name, avg_date_diff, COUNT(user_id) AS total_users FROM total_rides GROUP BY 1, 2) t2
              ON t1.country_name = t2.country_name AND t1.metric = t2.avg_date_diff
""")

df_dslr_incidents.head()

,country_name,metric,incidents,total_incidents,cum_sum_incidents,total_users,total_users_by_country,cum_sum_users
0,Brazil,1,1618,3932,1618,105309,2709493,105309
1,Brazil,2,473,3932,2091,170471,2709493,275780
2,Brazil,3,263,3932,2354,189540,2709493,465320
3,Brazil,4,204,3932,2558,187480,2709493,652800
4,Brazil,5,140,3932,2698,172380,2709493,825180


# Avatar / Name change

### Second launch for testing on Nicknames

In [45]:
df_nick_incidents = read_bq("""
WITH usernames AS (SELECT *
                   FROM (SELECT user_id,
                                username,
                                month,
                                modified,
                                COUNT(DISTINCT username) OVER (PARTITION BY user_id, month) AS metric
                         FROM (SELECT id                          AS user_id,
                                      username,
                                      DATE_TRUNC(modified, MONTH) AS month,
                                      MIN(modified)                  modified
                               FROM dwh-storage-327422.personal_data.dir_user_hst
                               WHERE start_dt_part >= '2024-01-01'
                                 AND DATE(modified) >= '2024-06-01'
                                 AND mode = 'client'
                               GROUP BY 1, 2, 3) _) __
                   ORDER BY 1),
     agg AS (SELECT user_id,
                    metric,
                    country_id,
                    country_name,
                    month,
                    ARRAY_TO_STRING(ARRAY_AGG(username), ', ') AS username
             FROM usernames t1
                      JOIN (SELECT t1.id,
                                   t1.city_id,
                                   t1.country_id,
                                   t2.country_name
                            FROM indriver-e6e40.ods_monolith.tbl_user t1
                                     JOIN indriver-bi.heap.vw_macroregion_mapping t2
                                          ON t1.city_id = t2.city_id
                            WHERE mode = 'client'
                              AND t1.country_id IN (11, 25, 12, 24, 22)) t2
                           ON t1.user_id = t2.id
             GROUP BY 1, 2, 3, 4, 5)
SELECT t1.*,
       SUM(incidents) OVER (PARTITION BY t1.country_id)                                                          AS total_incidents,
       total_users,
       SUM(total_users) OVER (PARTITION BY t1.country_name)                                                      AS total_users_by_country,
       SUM(total_users)
           OVER (PARTITION BY t1.country_id ORDER BY t1.metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum_users
FROM (SELECT t1.country_id,
             t1.country_name,
             t1.metric,
             COUNT(DISTINCT t2.redmine_id) AS incidents
      FROM agg t1
               JOIN (SELECT redmine_id,
                            incident_date,
                            vertical,
                            incident_type,
                            category,
                            city_id,
                            city_name,
                            pass_id
                     FROM indriver-bi.safety.vw_safety_incidents_detail
                     WHERE aggressor = 'Passenger'
                       AND incident_date >= '2024-06-01'
                       AND information_status = 'Confirmed') t2
                    ON t1.user_id = t2.pass_id
                        AND
                       t2.incident_date BETWEEN DATE(t1.month) AND DATE(t1.month) + INTERVAL 2 MONTH
      GROUP BY 1, 2, 3) t1
         JOIN (SELECT country_id, metric, COUNT(DISTINCT user_id) AS total_users FROM agg GROUP BY 1, 2) t2
              ON t1.country_id = t2.country_id
                  AND t1.metric = t2.metric
""")

df_nick_incidents.head()

,country_id,country_name,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users
0,25,Chile,0,5,1286,20302,1748744,20302
1,25,Chile,1,1149,1286,1689410,1748744,1709712
2,25,Chile,2,105,1286,37947,1748744,1747659
3,25,Chile,3,12,1286,877,1748744,1748536
4,25,Chile,4,7,1286,130,1748744,1748666


### Second launch for testing on Avatars

In [46]:
df_inc_avatars = read_bq("""
WITH avatar AS (SELECT id,
                       country_id,
                       city_id,
                       created,
                       modified,
                       DATE_TRUNC(modified, MONTH) AS month,
                       mode,
                       type,
                       avatarmedium,
                       gender,
                       username,
                FROM dwh-storage-327422.personal_data.dir_user_hst
                WHERE start_dt_part >= '2024-01-01'
                  AND created >= '2024-01-01'
                  AND mode = 'client'),
     cities AS (SELECT t1.id,
                       CAST(t1.created AS date)     AS reg_date,
                       DATE_TRUNC(t1.created, WEEK) AS month,
                       t1.mode,
                       t1.country_id,
                       t2.country_name,
                       t2.macroregion_name,
                       t1.city_id,
                       t2.city_name,
                       CASE
                           WHEN osversion LIKE '%android%' THEN 'android'
                           WHEN osversion LIKE '%ios%' THEN 'ios'
                           ELSE
                               NULL
                           END
                                                       os
                FROM indriver-e6e40.ods_monolith.tbl_user t1
                         JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                              ON t1.city_id = t2.city_id
                WHERE 1 = 1
                  AND t1.mode = 'client'
                QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.id ORDER BY t1.created DESC) = 1),
     avatar_change AS
         (SELECT id,
                 country_name,
                 country_id,
                 metric,
                 month,
                 MAX(modified)                           AS modified,
                 ARRAY_TO_STRING(ARRAY_AGG(photo), ', ') AS photo
          FROM (SELECT *,
                       COUNT(DISTINCT avatarmedium) OVER (PARTITION BY id, month) AS metric
                FROM (SELECT t1.id,
                             t1.month,
                             t2.country_name,
                             t2.country_id,
                             t2.city_id,
                             t2.city_name,
                             avatarmedium,
                             IF(avatarmedium IS NULL OR avatarmedium = '', 'no_photo', avatarmedium) AS photo,
                             modified
                      FROM avatar t1
                               JOIN cities t2
                                    ON t1.id = t2.id
                      WHERE DATE(modified) >= '2024-06-01' -- Window for avatar changing
                        AND t2.country_id IN (11, 25, 12, 24, 22)))
          GROUP BY 1, 2, 3, 4, 5)
SELECT t1.*,
       SUM(incidents) OVER (PARTITION BY t1.country_name)                                                        AS total_incidents,
       total_users,
       SUM(total_users) OVER (PARTITION BY t1.country_name)                                                      AS total_users_by_country,
       SUM(total_users)
           OVER (PARTITION BY t1.country_id ORDER BY t1.metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum_users
FROM (SELECT t1.country_name,
             t1.country_id,
             metric,
             COUNT(DISTINCT redmine_id) AS incidents
      FROM avatar_change t1
               JOIN (SELECT redmine_id,
                            incident_date,
                            vertical,
                            incident_type,
                            category,
                            city_id,
                            city_name,
                            pass_id
                     FROM indriver-bi.safety.vw_safety_incidents_detail
                     WHERE aggressor = 'Passenger'
                       AND incident_date >= '2024-01-01'
                       AND information_status = 'Confirmed') t2
                    ON t1.id = t2.pass_id
                        AND
                       t2.incident_date BETWEEN DATE(modified) AND DATE(modified) + INTERVAL 2 WEEK-- period for incidents
      GROUP BY 1, 2, 3) t1
         JOIN (SELECT country_id, metric, COUNT(DISTINCT id) AS total_users FROM avatar_change GROUP BY 1, 2) t2
              ON t1.country_id = t2.country_id
                  AND t1.metric = t2.metric
""")

df_inc_avatars.head()

,country_name,country_id,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users
0,Chile,25,0,201,595,244310,650984,244310
1,Chile,25,1,357,595,389676,650984,633986
2,Chile,25,2,29,595,15922,650984,649908
3,Chile,25,3,3,595,909,650984,650817
4,Chile,25,4,2,595,128,650984,650945


In [47]:
df_inc_avatars.query("country_name == 'Brazil'").sort_values('metric', ascending=True)

,country_name,country_id,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users
19,Brazil,11,0,1867,2757,2302736,3068818,2302736
20,Brazil,11,1,797,2757,729532,3068818,3032268
21,Brazil,11,2,79,2757,33406,3068818,3065674
22,Brazil,11,3,11,2757,2615,3068818,3068289
23,Brazil,11,4,1,2757,414,3068818,3068703
24,Brazil,11,5,2,2757,115,3068818,3068818


# Completed orders

In [48]:
df_completed_incidents = read_bq("""
WITH orders AS (SELECT user_type,
                       user_id,
                       t1.country_id,
                       country_name,
                       t1.city_id,
                       city_name,
                       metric_date_utc,
                       rides_count,
                       IF(rides_count > 0, 1, 0)                                                                                 AS has_ride,
                       LEAD(metric_date_utc)
                            OVER (PARTITION BY user_id ORDER BY metric_date_utc)                                                 AS next_ride_date,
                       SUM(rides_count)
                           OVER (PARTITION BY user_id ORDER BY metric_date_utc ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulitive_sum_of_rides
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                         JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                              ON t1.city_id = t2.city_id
                WHERE metric_date_utc >= '2024-01-01'
                  AND user_type = 'pass'
                  AND t1.city_id IN (4143, 4199, 4200, 4197, 4153, 4530, 4755, 4538, 4559, 4758,
                                     4519)),
     total_users AS (SELECT city_id, user_id, SUM(rides_count) AS completed_orders
                     FROM orders
                     GROUP BY 1, 2)
SELECT t1.*,
       SUM(incidents) OVER (PARTITION BY t1.city_id)                                                               AS total_incidents,
       SUM(incidents)
           OVER (PARTITION BY t1.country_name ORDER BY t1.metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum_incidents,
       t2.total_users,
       SUM(total_users) OVER (PARTITION BY t1.country_name)                                                        AS total_users_by_country,
       SUM(total_users)
           OVER (PARTITION BY t1.city_id ORDER BY metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)         AS cum_sum_users
FROM (SELECT t1.city_id,
             t1.city_name,
             t1.country_name,
             cumulitive_sum_of_rides AS metric,
             COUNT(user_id)          AS incidents
      FROM orders t1
               JOIN (SELECT *
                     FROM indriver-bi.safety.vw_safety_incidents_detail
                     WHERE aggressor = 'Passenger'
                       AND incident_date >= '2024-01-01'
                       AND information_status = 'Confirmed') t2
                    ON t1.user_id = t2.pass_id
                        AND t2.incident_date = metric_date_utc
      GROUP BY 1, 2, 3, 4) t1
         JOIN (SELECT completed_orders, city_id, COUNT(user_id) AS total_users
               FROM total_users
               GROUP BY 1, 2) t2
              ON t1.city_id = t2.city_id AND t1.metric = t2.completed_orders
""")

df_completed_incidents.head()

df_completed_incidents['cum_incidents'] = df_completed_incidents['cum_sum_incidents'] / df_completed_incidents['total_incidents'] * 100

df_completed_incidents['cum_impact'] = df_completed_incidents['cum_sum_users'] / df_completed_incidents['total_users_by_country'] * 100

In [49]:
df_completed_incidents.query("city_id in (4143, 4199, 4200, 4197, 4153, 4530, 4755, 4538, 4559, 4758,4519)").sort_values('metric', ascending=True).head(30)

,city_id,city_name,country_name,metric,incidents,total_incidents,cum_sum_incidents,total_users,total_users_by_country,cum_sum_users,cum_incidents,cum_impact
0,4519,Vitoria,Brazil,0,31,174,283,25054,978976,25054,162.64,2.56
151,4153,Salvador,Brazil,0,170,598,453,69369,978976,69369,75.75,7.09
365,4200,Santiago,Chile,0,343,930,343,112427,442603,112427,36.88,25.40
457,4199,Lima,Peru,0,415,2622,415,423109,3923806,423109,15.83,10.78
693,4530,Brasília,Brazil,0,42,359,252,53130,978976,53130,70.19,5.43
87,4755,Porto Alegre,Brazil,0,84,408,210,63029,978976,63029,51.47,6.44
775,4197,Bogota,Colombia,0,379,1388,379,418448,2225821,418448,27.31,18.80
891,4538,Campo Grande,Brazil,0,18,173,471,26007,978976,26007,272.25,2.66
32,4559,Campinas,Brazil,0,72,229,126,21345,978976,21345,55.02,2.18
949,4758,Santos,Brazil,0,54,77,54,12825,978976,12825,70.13,1.31


# Negative reviews

In [50]:
df_inc_reviews = read_bq("""
WITH review AS (SELECT t1.target_id,
                       t1.country_id,
                       t1.city_id,
                       t2.country_name,
                       DATE_TRUNC(t1.created_date_part, MONTH) AS month,
                       COUNT(t1.rating)                        AS metric,
                       AVG(t1.rating)                          AS avg_rating,
                       MAX(t1.created_date_part)               AS last_review_date
                FROM indriver-e6e40.ods_review.tbl_review t1
                         JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                              ON t1.city_id = t2.city_id
                WHERE visibility = 'passenger'
                  AND t1.country_id IN (11, 25, 12, 24, 22)
                  AND created_date_part >= '2024-06-01'
                  AND type = 'new_order'
                GROUP BY 1, 2, 3, 4, 5)
SELECT t1.*,
       SUM(incidents) OVER (PARTITION BY t1.country_id)                                                          AS total_incidents,
       total_users,
       SUM(total_users) OVER (PARTITION BY t1.country_id)                                                        AS total_users_by_country,
       SUM(total_users)
           OVER (PARTITION BY t1.country_id ORDER BY t1.metric ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum_users
FROM (SELECT t1.country_id,
             t1.country_name,
             t1.metric,
             COUNT(redmine_id) AS incidents
      FROM (SELECT * FROM review WHERE avg_rating <= 2) t1
               JOIN (SELECT redmine_id,
                            incident_date,
                            vertical,
                            incident_type,
                            category,
                            city_id,
                            city_name,
                            pass_id
                     FROM indriver-bi.safety.vw_safety_incidents_detail
                     WHERE aggressor = 'Passenger'
                       AND incident_date >= '2024-06-01'
                       AND information_status = 'Confirmed') t2
                    ON t1.target_id = t2.pass_id
                        AND t2.incident_date BETWEEN t1.last_review_date AND t1.last_review_date + INTERVAL 1 MONTH
      GROUP BY 1, 2, 3) t1
         JOIN (SELECT country_id, metric, COUNT(DISTINCT target_id) AS total_users
               FROM review
               WHERE avg_rating <= 2
               GROUP BY 1, 2) t2
              ON t1.country_id = t2.country_id
                  AND t1.metric = t2.metric
""")

df_inc_reviews.head()

,country_id,country_name,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users
0,25,Chile,1,211,266,36655,39027,36655
1,25,Chile,2,35,266,1895,39027,38550
2,25,Chile,3,8,266,223,39027,38773
3,25,Chile,4,3,266,189,39027,38962
4,25,Chile,5,3,266,40,39027,39002


In [51]:
df_inc_reviews = df_inc_reviews.rename(columns={'country_name':'region'})

# Drivers complaints

In [83]:
df_complaints_scale = read_bq("""
WITH appeals AS (SELECT DATE(t1.created_at)                                                        AS created_at,
                        CAST(`indriver-e6e40.de_functions.convert_uuid_to_id`(target_uuid) AS INT) AS user_id,
                        t1.text,
                        t1.initiator,
                        t1.country_id,
                        t2.country_name,
                        t1.city_id,
                        t2.city_name,
                 FROM dwh-storage-327422.ods_review.appeal t1
                          JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                               ON t1.city_id = t2.city_id
                 WHERE t1.initiator = 'contractor'
                   AND t1.created_at >= '2024-10-01'),
     total_rides AS (SELECT t1.country_id
                          , country_name
--                           , metric_date_utc
                          , SUM(rides_count) AS rides
                     FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                              JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                                   ON t1.city_id = t2.city_id
                     WHERE metric_date_utc >= '2024-10-01'
                       AND user_type = 'pass'
                     GROUP BY 1
                            , 2)
        ,
     incidents AS (SELECT redmine_id
                        , incident_date
                        , vertical
                        , aggressor
                        , incident_type
                        , category
                        , city_id
                        , city_name
                        , pass_id
                        , driver_id
                   FROM indriver-bi.safety.vw_safety_incidents_detail
                   WHERE aggressor = 'Passenger'
                     AND incident_date >= '2024-10-01'
                     AND information_status = 'Confirmed')
SELECT DISTINCT t1.country_id,
                t1.country_name,
                t1.user_id,
                t3.redmine_id,
                t2.rides
FROM appeals t1
         JOIN total_rides t2
              ON t1.country_id = t2.country_id
--                   AND t1.created_at = t2.metric_date_utc
         LEFT JOIN incidents t3
                   ON t1.user_id = t3.pass_id
                       AND t3.incident_date >= t1.created_at
""")


# df_complaints_scale['impact'] = (df_complaints_scale['appealed'] / df_complaints_scale['rides']).round(5)
# df_complaints_scale['impact_%'] = df_complaints_scale['appealed'] / df_complaints_scale['rides'] * 100

# df_complaints_scale['monthly'] = pd.to_datetime(df_complaints_scale['monthly'])

df_complaints_scale.head()

,country_id,country_name,user_id,redmine_id,rides
0,13,Argentina,172639941,None,422328
1,85,Malaysia,126819379,None,2589679
2,19,Thailand,98960367,None,999960
3,76,Nicaragua,86335522,None,949550
4,85,Malaysia,263105446,None,2589679


In [88]:
impact_of_certain_countries = df_complaints_scale.groupby(['country_name'], as_index=False)[['user_id', 'redmine_id', 'rides']].agg({'user_id':'count',
                                                                                                                                     'redmine_id':'count',
                                                                                                                                     'rides':'max',}) \
                    .query("country_name in ('Brazil', 'Chile', 'Mexico', 'South Africa', 'Colombia', 'Peru')") \
                    .sort_values(['country_name', 'user_id'], ascending=False)

impact_of_certain_countries['impact_%'] = impact_of_certain_countries['user_id'] / impact_of_certain_countries['rides'] * 100

impact_of_certain_countries['ir'] = impact_of_certain_countries['redmine_id'] / impact_of_certain_countries['rides'] * 100000

impact_of_certain_countries

,country_name,user_id,redmine_id,rides,impact_%,ir
37,South Africa,51779,320,5163190,1.00,6.20
35,Peru,189487,429,35456299,0.53,1.21
27,Mexico,150061,550,33484955,0.45,1.64
7,Colombia,204706,447,32376458,0.63,1.38
6,Chile,51825,305,9280204,0.56,3.29
5,Brazil,161908,1256,19521812,0.83,6.43


# Device has been changed by user

In [79]:
df_device_change = read_bq("""
WITH rides AS (SELECT t1.country_id,
                      country_name,
                      metric_date_utc,
                      SUM(rides_count) AS rides
               FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                        JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                             ON t1.city_id = t2.city_id
               WHERE metric_date_utc = '2024-10-01'
                 AND user_type = 'driver'
                 AND t1.country_id IN (11, 25, 12, 24, 22)
               GROUP BY 1, 2, 3),
     incidents AS (SELECT redmine_id,
                          incident_date,
                          vertical,
                          aggressor,
                          incident_type,
                          category,
                          city_id,
                          city_name,
                          pass_id,
                          driver_id
                   FROM indriver-bi.safety.vw_safety_incidents_detail
                   WHERE aggressor = 'Passenger'
                     AND incident_date >= '2024-10-01'
                     AND information_status = 'Confirmed'),
     change_device AS (SELECT user_id, country_id, session_dt_part, COUNT(shield_id) AS cnt_devices
                       FROM (SELECT DISTINCT fl.user_id, fl.country_id, session_dt_part, shield_id
                             FROM indriver-e6e40.antifraud.fingerprint_log fl
                                      JOIN indriver-e6e40.antifraud.shield_fingerprints sf USING (fingerprint_id)
                             WHERE event_dt_part = '2024-10-01'
                               AND session_dt_part = '2024-10-01'
                               AND shield_id != 'NOT_FOUND'
                               AND fl.user_id IS NOT NULL
                               AND fl.country_id IN (11, 25, 12, 24, 22)
                             QUALIFY ROW_NUMBER() OVER (PARTITION BY fl.user_id, shield_id ORDER BY sf.timestamp) = 1)
                       GROUP BY 1, 2, 3)
SELECT t1.user_id,
       t1.country_id,
       loc.name,
       t1.session_dt_part,
       t1.cnt_devices,
       t3.rides,
       t2.redmine_id,
       t2.incident_date,
       t2.pass_id
FROM change_device t1
         JOIN indriver-e6e40.ods_monolith.tbl_country loc
              ON t1.country_id = loc.id
         LEFT JOIN incidents t2
                   ON t1.user_id = t2.pass_id
                       AND t2.incident_date >= session_dt_part
         LEFT JOIN rides t3
                   ON t1.country_id = t3.country_id
                       AND t1.session_dt_part = t3.metric_date_utc
""")

df_device_change.head(10)

# add incidents to add columns

,user_id,country_id,name,session_dt_part,cnt_devices,rides,redmine_id,incident_date,pass_id
0,18080726,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
1,18900541,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
2,20293827,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
3,20952262,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
4,21849181,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
5,22662026,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
6,23822846,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
7,24136675,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
8,24983667,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>
9,25566028,11,Brazil,2024-10-01,1,305068,None,NaT,<NA>


In [81]:
df_device_change_agg = df_device_change.query("cnt_devices > 1").groupby(['name', 'session_dt_part'], as_index=False)[['user_id', 'redmine_id', 'rides']] \
                        .agg({'user_id':'nunique',
                            'redmine_id':'count',
                            'rides':'max'})

df_device_change_agg['share_of_users'] = df_device_change_agg['user_id'] / df_device_change_agg['rides'] * 100

df_device_change_agg['IR'] = df_device_change_agg['redmine_id'] / df_device_change_agg['rides'] * 100000

df_device_change_agg

,name,session_dt_part,user_id,redmine_id,rides,share_of_users,IR
0,Brazil,2024-10-01,1874,0,305068,0.61,0.00
1,Chile,2024-10-01,533,0,142611,0.37,0.00
2,Colombia,2024-10-01,2007,1,478395,0.42,0.21
3,Mexico,2024-10-01,1335,2,425787,0.31,0.47
4,Peru,2024-10-01,1946,0,515487,0.38,0.00


------
# Confusion matrix

In [32]:
def confusion_matrix_by_cities(df, completed_orders = False):

    """
    Here is function to create a confusion matrix with TPR and FNR by using DataFrame\n
    df: dataframe with certain columns
    """

    if completed_orders == False:

        groupby = df.groupby(['region', 'metric'], as_index=False)[['incidents', 
                                                                    'total_incidents', 
                                                                    'total_users', 
                                                                    'total_users_by_country',
                                                                    'cum_sum_users']].agg({'incidents':'sum', 
                                                                                            'total_incidents':'max', 
                                                                                            'total_users':'max', 
                                                                                            'total_users_by_country':'max',
                                                                                            'cum_sum_users':'max'})
        
        groupby['TPR'] = groupby['incidents'] / groupby['total_incidents'] * 100
        groupby['FNR'] = 100 - (groupby['incidents'] / groupby['total_incidents']*100)
        groupby['Impact'] = groupby['total_users'] / groupby['total_users_by_country'] * 100
        groupby['Regarding the users / IR'] = groupby['incidents'] / groupby['total_users'] * 100

        groupby = groupby.round(2)

        columns = ['<b>Region</b>', '<b>Metric (Threshold)</b>', '<b>Incidents</b>', '<b>Users by threshold</b>', '<b>Total users by region</b>', '<b>TPR, %</b>', '<b>FNR, %</b>', '<b>Impact, %</b>', '<b>Regarding the users / IR</b>']

        header = 'grey'
        row = 'lightgrey'

        fig = go.Figure(data=[go.Table(
            header=dict(values=list(columns),
                        line_color='darkslategray',
                        fill_color=header,
                        align='left',
                        font=dict(color='white', size=14),
                        height=20),
            cells=dict(values=[groupby.region, groupby.metric, groupby.incidents, groupby.total_users, groupby.total_users_by_country, groupby['TPR'], groupby['FNR'], groupby['Impact'], groupby['Regarding the users / IR']],
                    align='left',
                    line_color='darkslategray',
                    fill_color=row,
                    font_size=12,
                    height=25))
        ])

        fig.update_layout(width=1400, height=1000)

        fig.show()
    
    elif completed_orders == True:

        groupby = df.groupby(['region', 'metric'], as_index=False)[['incidents', 
                                                                    'total_incidents', 
                                                                    'total_users', 
                                                                    'total_users_by_country',
                                                                    'cum_sum_users',
                                                                    'cum_sum_incidents']].agg({'incidents':'sum', 
                                                                                            'total_incidents':'max', 
                                                                                            'total_users':'max', 
                                                                                            'total_users_by_country':'max',
                                                                                            'cum_sum_users':'max',
                                                                                            'cum_sum_incidents':'max'})
        
        groupby['TPR'] = groupby['incidents'] / groupby['total_incidents'] * 100
        groupby['FNR'] = 100 - (groupby['incidents'] / groupby['total_incidents']*100)
        groupby['Impact'] = groupby['total_users'] / groupby['total_users_by_country'] * 100
        groupby['Regarding the users / IR'] = groupby['incidents'] / groupby['total_users'] * 100
        groupby['Cum_incidents'] = groupby['cum_sum_incidents'] / groupby['total_incidents'] * 100
        groupby['Cum_impact'] = groupby['cum_sum_users'] / groupby['total_users_by_country'] * 100

        groupby = groupby.round(2)

        columns = ['<b>Region</b>', '<b>Metric (Threshold)</b>', '<b>Cumalitive incidents</b>', '<b>Cumalitive users by threshold</b>', '<b>Total users by region</b>', 
                   '<b>TPR, %</b>', '<b>Cumalitive TPR, %</b>', '<b>FNR, %</b>', '<b>Impact, %</b>', '<b>Cumalitive impact, %</b>', '<b>Regarding the users / IR</b>']

        header = 'grey'
        row = 'lightgrey'

        fig = go.Figure(data=[go.Table(
            header=dict(values=list(columns),
                        line_color='darkslategray',
                        fill_color=header,
                        align='left',
                        font=dict(color='white', size=14),
                        height=20),
            cells=dict(values=[groupby.region, groupby.metric, groupby.cum_sum_incidents, groupby.cum_sum_users, groupby.total_users_by_country, groupby['TPR'], groupby['Cum_incidents'], groupby['FNR'], groupby['Impact'], groupby['Cum_impact'], groupby['Regarding the users / IR']],
                    align='left',
                    line_color='darkslategray',
                    fill_color=row,
                    font_size=12,
                    height=25))
        ])

        fig.update_layout(width=1400, height=1000)

        fig.show()



    return groupby

In [65]:
df_dslr_incidents.head()

df_dslr_incidents=df_dslr_incidents.rename(columns={'country_name':'region'})

confusion_matrix_by_cities(df_dslr_incidents)

,region,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users,TPR,FNR,Impact,Regarding the users / IR
0,Brazil,1,1618,3932,105309,2709493,105309,41.15,58.85,3.89,1.54
1,Brazil,2,473,3932,170471,2709493,275780,12.03,87.97,6.29,0.28
2,Brazil,3,263,3932,189540,2709493,465320,6.69,93.31,7.00,0.14
3,Brazil,4,204,3932,187480,2709493,652800,5.19,94.81,6.92,0.11
4,Brazil,5,140,3932,172380,2709493,825180,3.56,96.44,6.36,0.08
...,...,...,...,...,...,...,...,...,...,...,...
422,Peru,115,1,1389,625,3267470,3265045,0.07,99.93,0.02,0.16
423,Peru,116,1,1389,584,3267470,3265629,0.07,99.93,0.02,0.17
424,Peru,120,1,1389,650,3267470,3266279,0.07,99.93,0.02,0.15
425,Peru,124,2,1389,566,3267470,3266845,0.14,99.86,0.02,0.35


In [60]:
df_nick_incidents.head()

df_nick_incidents=df_nick_incidents.rename(columns={'country_name':'region'})

confusion_matrix_by_cities(df_nick_incidents)

,region,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users,TPR,FNR,Impact,Regarding the users / IR
0,Brazil,0,29,6671,264978,6869066,264978,0.43,99.57,3.86,0.01
1,Brazil,1,5885,6671,6418684,6869066,6683662,88.22,11.78,93.44,0.09
2,Brazil,2,615,6671,181069,6869066,6864731,9.22,90.78,2.64,0.34
3,Brazil,3,91,6671,3532,6869066,6868263,1.36,98.64,0.05,2.58
4,Brazil,4,28,6671,520,6869066,6868783,0.42,99.58,0.01,5.38
5,Brazil,5,13,6671,166,6869066,6868949,0.19,99.81,0.00,7.83
6,Brazil,6,7,6671,89,6869066,6869038,0.10,99.90,0.00,7.87
7,Brazil,7,3,6671,28,6869066,6869066,0.04,99.96,0.00,10.71
8,Chile,0,5,1286,20302,1748744,20302,0.39,99.61,1.16,0.02
9,Chile,1,1149,1286,1689410,1748744,1709712,89.35,10.65,96.61,0.07


In [62]:
df_inc_avatars.head()

df_inc_avatars=df_inc_avatars.rename(columns={'country_name':'region'})

confusion_matrix_by_cities(df_inc_avatars)

,region,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users,TPR,FNR,Impact,Regarding the users / IR
0,Brazil,0,1867,2757,2302736,3068818,2302736,67.72,32.28,75.04,0.08
1,Brazil,1,797,2757,729532,3068818,3032268,28.91,71.09,23.77,0.11
2,Brazil,2,79,2757,33406,3068818,3065674,2.87,97.13,1.09,0.24
3,Brazil,3,11,2757,2615,3068818,3068289,0.40,99.60,0.09,0.42
4,Brazil,4,1,2757,414,3068818,3068703,0.04,99.96,0.01,0.24
5,Brazil,5,2,2757,115,3068818,3068818,0.07,99.93,0.00,1.74
6,Chile,0,201,595,244310,650984,244310,33.78,66.22,37.53,0.08
7,Chile,1,357,595,389676,650984,633986,60.00,40.00,59.86,0.09
8,Chile,2,29,595,15922,650984,649908,4.87,95.13,2.45,0.18
9,Chile,3,3,595,909,650984,650817,0.50,99.50,0.14,0.33


In [63]:
df_completed_incidents.head()

df_completed_incidents=df_completed_incidents.query("city_id in (4143, 4199, 4200, 4197, 4153, 4530, 4755, 4538, 4559, 4758,4519)").rename(columns={'city_name':'region'})

confusion_matrix_by_cities(df_completed_incidents, completed_orders=True)

,region,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users,cum_sum_incidents,TPR,FNR,Impact,Regarding the users / IR,Cum_incidents,Cum_impact
0,Bogota,0,379,1388,418448,2225821,418448,379,27.31,72.69,18.80,0.09,27.31,18.80
1,Bogota,1,196,1388,435512,2225821,853960,575,14.12,85.88,19.57,0.05,41.43,38.37
2,Bogota,2,128,1388,240846,2225821,1094806,703,9.22,90.78,10.82,0.05,50.65,49.19
3,Bogota,3,68,1388,155897,2225821,1250703,771,4.90,95.10,7.00,0.04,55.55,56.19
4,Bogota,4,58,1388,115702,2225821,1366405,829,4.18,95.82,5.20,0.05,59.73,61.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Vitoria,48,1,174,110,978976,77271,1832,0.57,99.43,0.01,0.91,"1,052.87",7.89
949,Vitoria,57,1,174,79,978976,77350,1878,0.57,99.43,0.01,1.27,"1,079.31",7.90
950,Vitoria,64,1,174,52,978976,77402,1899,0.57,99.43,0.01,1.92,"1,091.38",7.91
951,Vitoria,113,1,174,14,978976,77416,1964,0.57,99.43,0.00,7.14,"1,128.74",7.91


In [64]:
df_inc_reviews.head()

df_inc_reviews=df_inc_reviews.rename(columns={'country_name':'region'})

confusion_matrix_by_cities(df_inc_reviews)

,region,metric,incidents,total_incidents,total_users,total_users_by_country,cum_sum_users,TPR,FNR,Impact,Regarding the users / IR
0,Brazil,1,763,895,130590,138597,130590,85.25,14.75,94.22,0.58
1,Brazil,2,101,895,6501,138597,137091,11.28,88.72,4.69,1.55
2,Brazil,3,13,895,693,138597,137784,1.45,98.55,0.50,1.88
3,Brazil,4,15,895,639,138597,138423,1.68,98.32,0.46,2.35
4,Brazil,5,2,895,135,138597,138558,0.22,99.78,0.10,1.48
5,Brazil,6,1,895,39,138597,138597,0.11,99.89,0.03,2.56
6,Chile,1,211,266,36655,39027,36655,79.32,20.68,93.92,0.58
7,Chile,2,35,266,1895,39027,38550,13.16,86.84,4.86,1.85
8,Chile,3,8,266,223,39027,38773,3.01,96.99,0.57,3.59
9,Chile,4,3,266,189,39027,38962,1.13,98.87,0.48,1.59
